In [ ]:
import pandas as pd
from google.cloud import storage
from io import BytesIO
from scipy.sparse.linalg import svds
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
client = storage.Client()
bucket_name = "meet-fresh-recommendation-system-data"
file_name = "meetfresh_customer_item_rating_matrix.csv"
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_name)
content = blob.download_as_string()
matrix = pd.read_csv(BytesIO(content))
matrix = matrix.rename(columns={"Unnamed: 0": "customer_id"}).astype({'customer_id': int})

In [ ]:
matrix.head()

,customer_id,Tofu Pudding,Rice Balls,Boba,Mini Q (Mini Taro Ball),Red Beans,Grass Jelly,Peanuts,Black Sugar Boba,Melon Jelly,...,Potaro Balls,Caramel Pudding,Sesame Rice Balls,Taro Balls,Almond Pudding,Taro Paste,Q Mochi,Coco Mochi,Sago,Almond Flakes
0,49237,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN
1,172644,1.0,NaN,3.0,3.0,2.0,3.0,3.0,NaN,NaN,...,2.0,NaN,3.0,1.0,2.0,2.0,NaN,NaN,3.0,NaN
2,39930,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,1.0,3.0,NaN,NaN,NaN
3,207442,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,78835,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
matrix.columns

Index(['customer_id', 'Tofu Pudding', 'Rice Balls', 'Boba',
       'Mini Q (Mini Taro Ball)', 'Red Beans', 'Grass Jelly', 'Peanuts',
       'Black Sugar Boba', 'Melon Jelly', 'Lychee Jelly', 'Ice Cream', 'Taro',
       'Potaro Balls', 'Caramel Pudding', 'Sesame Rice Balls', 'Taro Balls',
       'Almond Pudding', 'Taro Paste', 'Q Mochi', 'Coco Mochi', 'Sago',
       'Almond Flakes'],
      dtype='object')

In [ ]:
matrix_train, matrix_test = train_test_split(matrix,
                                   test_size=0.20,
                                   random_state=42)
print(matrix_train.shape)
print(matrix_test.shape)

(124475, 23)
(31119, 23)


In [ ]:
train_df = matrix_train.set_index("customer_id")
train_df = train_df.fillna(0)
test_df = matrix_test.set_index("customer_id")
test_df = test_df.fillna(0)
train_df.head()

,Tofu Pudding,Rice Balls,Boba,Mini Q (Mini Taro Ball),Red Beans,Grass Jelly,Peanuts,Black Sugar Boba,Melon Jelly,Lychee Jelly,...,Potaro Balls,Caramel Pudding,Sesame Rice Balls,Taro Balls,Almond Pudding,Taro Paste,Q Mochi,Coco Mochi,Sago,Almond Flakes
customer_id,,,,,,,,,,,,,,,,,,,,,
40920,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
175469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117348,0.0,1.0,2.0,2.0,1.0,3.0,0.0,0.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60973,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146008,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
U, sigma, Vt = svds(train_df, k = 5)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = train_df.columns, index=train_df.index)

In [ ]:
sorted_user_predictions = preds_df.loc[40920].sort_values(ascending=False)
sorted_user_predictions

Taro Balls                 1.999372
Grass Jelly                1.998946
Rice Balls                 0.024642
Tofu Pudding               0.020538
Caramel Pudding            0.019857
Melon Jelly                0.016761
Potaro Balls               0.016297
Sesame Rice Balls          0.013750
Almond Pudding             0.011453
Peanuts                    0.011206
Q Mochi                    0.006941
Black Sugar Boba           0.005378
Lychee Jelly               0.004643
Taro                       0.002816
Almond Flakes              0.001266
Sago                       0.000659
Coco Mochi                -0.000171
Boba                      -0.001617
Ice Cream                 -0.006235
Red Beans                 -0.006405
Taro Paste                -0.012664
Mini Q (Mini Taro Ball)   -0.024417
Name: 40920, dtype: float64

In [ ]:
sorted_user_predictions["Boba"]

-0.0016170149488952878

In [ ]:
def recommend_items(predictions_df, userID, num_recommendations=5):

    # Get and sort the user's predictions
    sorted_user_predictions = predictions_df.loc[userID].sort_values(ascending=False)
    res = []
    for item in sorted_user_predictions.index:
        if len(res) >= num_recommendations:
            break
        if train_df.loc[userID, item] == 0:
            res.append((sorted_user_predictions[item], item))
    return res

reco = recommend_items(preds_df, 40920, 5)
reco

[(0.024641623691538366, 'Rice Balls'),
 (0.020537986020375366, 'Tofu Pudding'),
 (0.01985744062605331, 'Caramel Pudding'),
 (0.01676083449538562, 'Melon Jelly'),
 (0.016296669486365323, 'Potaro Balls')]

## Modeling

In [ ]:
class CFModel:

    def __init__(self, items_df=None):
        self.MODEL_NAME = 'Collaborative Filtering #1'
        self._preds_df = None
        self._train = items_df


    def get_model_name(self):
        return self.MODEL_NAME


    def get_svd(self):
        if self._preds_df is None:
            U, sigma, Vt = svds(self._train, k = 5)
            sigma = np.diag(sigma)
            all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
            preds_df = pd.DataFrame(all_user_predicted_ratings, columns = self._train.columns, index=self._train.index)
            self._preds_df = preds_df
        return True


    def recommend_items(self, user_id, topk=5):
        self.get_svd()
        sorted_user_predictions = self._preds_df.loc[user_id].sort_values(ascending=False)
        res = []
        for item in sorted_user_predictions.index:
            if len(res) >= topk:
                break
            if self._train.loc[user_id, item] == 0:
                res.append((sorted_user_predictions[item], item))
        return res


cf_model = CFModel(train_df)
cf_model.recommend_items(user_id=40920, topk=5)

[(0.024641623691538554, 'Rice Balls'),
 (0.02053798602037546, 'Tofu Pudding'),
 (0.019857440626053123, 'Caramel Pudding'),
 (0.016760834495385887, 'Melon Jelly'),
 (0.01629666948636589, 'Potaro Balls')]

## Evaluation

In [ ]:
class Evaluator:
    EVALUATOR_NAME = "eva"

    def __init__(self, model, test_data):
        self._test = test_data
        self.model = model

    def draw_user(self):
        user = self._test.sample(1, random_state=43)
        uid = user.index[0]
        user = user.iloc[0, :].sort_values(ascending=False)
        return uid, user

    def generate_score(self, topk=5):
        uid, user = self.draw_user()
        pred = self.model.recommend_items(uid, topk)
        obt = 0
        # pred = [(0, item) for score, item in pred]
        # pred = [(3, item) for score, item in pred]
        print(user)
        print(pred)
        for score, item in pred:
            target = user[item]
            obt += 1 - abs(score-target) / 3

        # TODO: another evaluation: ADD RECALL
        # TODO: should do more calculations and take average (MAP)

        ret = obt/topk
        print("The model \'{}\' has {:.2f}% precision".format(self.model.MODEL_NAME, ret * 100))
        return ret

evaluator = Evaluator(cf_model, train_df) # How to use test set?
g = evaluator.generate_score()


Taro                       3.0
Potaro Balls               3.0
Boba                       3.0
Ice Cream                  3.0
Sago                       0.0
Coco Mochi                 0.0
Q Mochi                    0.0
Taro Paste                 0.0
Almond Pudding             0.0
Taro Balls                 0.0
Sesame Rice Balls          0.0
Caramel Pudding            0.0
Tofu Pudding               0.0
Rice Balls                 0.0
Lychee Jelly               0.0
Melon Jelly                0.0
Black Sugar Boba           0.0
Peanuts                    0.0
Grass Jelly                0.0
Red Beans                  0.0
Mini Q (Mini Taro Ball)    0.0
Almond Flakes              0.0
Name: 14868, dtype: float64
[(1.5967230123041145, 'Mini Q (Mini Taro Ball)'), (1.4026677183666385, 'Taro Paste'), (0.6013658796358275, 'Sesame Rice Balls'), (0.48659831183052055, 'Melon Jelly'), (0.46170672854562717, 'Rice Balls')]
The model 'Collaborative Filtering #1' has 69.67% precision


In [ ]:
class Evaluator:
    EVALUATOR_NAME = "MAP"

    def __init__(self, model, test_data):
        self._test = test_data
        self.model = model

    def draw_user(self):
        user = self._test.sample(1, random_state=42)
        uid = user.index[0]
        user = user.iloc[0, :].sort_values(ascending=False)
        return uid, user

    def generate_score(self, topk=5):
        tot_obt = 0
        sample_size = 100
        for index, row in self._test.sample(sample_size, random_state=123).iterrows():
            user = self._test.loc[index, :]
            uid = user.name
            pred = self.model.recommend_items(uid, topk)
            obt = 0
            # pred = [(0, item) for score, item in pred]
            # pred = [(3, item) for score, item in pred]
            for score, item in pred:
                target = user[item]
                obt += 1 - abs(score-target) / 3
            # TODO: another evaluation: ADD RECALL
            tot_obt += obt

        ret = obt/topk
        print("The model \'{}\' has {:.2f}% Precision".format(self.model.MODEL_NAME, ret * 100))
        print("The model \'{}\' has {:.2f}% Average Precision".format(self.model.MODEL_NAME, tot_obt / topk / sample_size * 100))
        return ret

evaluator = Evaluator(cf_model, train_df) # How to use test set?
g = evaluator.generate_score()

The model 'Collaborative Filtering #1' has 86.76% Precision
The model 'Collaborative Filtering #1' has 91.49% Average Precision
